In [2]:
import pandas as pd
from os.path import join, expanduser

# Load Base Data

In [46]:
df = pd.read_csv('/d/PBJ_Daily_Nurse_Staffing_Q2_2024.csv', encoding='latin1', low_memory=False)
df.columns = [x.lower() for x in df.columns]
BASE_COLS = ['provnum', 'city', 'state', 'county_name', 'county_fips', 'cy_qtr', 'workdate']
assert df[BASE_COLS].drop_duplicates().shape[0] / len(df) == 1

In [49]:
df.columns = [x.lower() for x in df.columns]
print([x for x in df.columns if not x.startswith('hrs_')])

['provnum', 'provname', 'city', 'state', 'county_name', 'county_fips', 'cy_qtr', 'workdate', 'mdscensus']


In [53]:
cols_hrs = [x for x in df.columns if x.startswith('hrs_') and not x.endswith('_emp') and not x.endswith('_ctr')]
print(cols_hrs)

['hrs_rndon', 'hrs_rnadmin', 'hrs_rn', 'hrs_lpnadmin', 'hrs_lpn', 'hrs_cna', 'hrs_natrn', 'hrs_medaide']


In [87]:
gr = df.groupby('state')[cols_hrs].sum()
idx = gr.sum(axis=1).sort_values().index.tolist()
gr = gr.reindex(idx)
gr = gr.reset_index()

In [101]:
roletype_names = {
    'hrs_rndon': 'RN Director of Nursing',
    'hrs_rnadmin': 'RN with Admin Duties',
    'hrs_rn': 'Registered Nurse',
    'hrs_lpnadmin': 'LPN with Admin Duties',
    'hrs_lpn': 'Licensed Practical Nurse',
    'hrs_cna': 'Certified Nursing Assistant',
    'hrs_natrn': 'Nurse Aide Trainee',
    'hrs_medaide': 'Medication Aide'
}

In [114]:
import plotly.express as px

colors = ['#1F7ABE', '#F0C277', '#C48646', '#D80A0C', '#000000', '#0E1B35', '#635D70', '#463F1A']

roletype_order = [
    'hrs_cna', 
    'hrs_lpn', 
    'hrs_rn', 
    'hrs_medaide',
    'hrs_rnadmin', 
    'hrs_lpnadmin', 
    'hrs_rndon', 
    'hrs_natrn', 
]

color_map = dict(zip(roletype_order, colors[:len(roletype_order)]))
color_map = {roletype_names[k]:v for k,v in color_map.items()}

df_melted = gr.melt(id_vars=['state'], 
                  value_vars=roletype_order,
                  var_name='category', value_name='hours')

df_melted['category'] = df_melted['category'].map(roletype_names)

fig = px.bar(df_melted, 
             y='state', 
             x='hours', 
             color='category',
             orientation='h',
             color_discrete_map=color_map)

fig.write_html('../images/hours-by-state.html')